In [ ]:
"""I Target： Here we learn how to make dataset

II Definition: In Keras, Dataset is not actual a images-set, it's a defined class, if you want to design your own
 Dataset, you should inherit from torch.utils.data.Dataset. Actually, you can get images and labels by __getitem__,
 where they store in torch.utils.data.Dataset.
    
III Instances:
    There are 3 ways to load and preprocess image dataset 
    1.0 build tf.data.Dataset ,  by random x,y sample
    2.0 use high-level Keras preprocessing utilities(e.g. tf.keras.utils.image_dataset_from_directory)
        and layers (e.g. tf.keras.layers.Rescaling)
    3.0 down load tensorflow datasets


IV Compare 2 then Generalize

V Test in New instance 
"""

In [1]:
import tensorflow as tf
import numpy as np

In [19]:
def build_dataset_by_random_sample():
    """1.0&1.1 build tf.data.Dataset, by  random x&y 
        quickest way to build tf.data.Dataset is from tf.data.Dataset.from_tensor_slices
    """
    x = np.linspace(-1, 1, 10)
    y = x**2

    ds_without_labels = tf.data.Dataset.from_tensor_slices(x)  # ,y
    print(f"ds_without_labels: {ds_without_labels}")

    dataset  = tf.data.Dataset.from_tensor_slices((x,y))  
    print(f"dataset with labels:{dataset}")

    # or you can zip x and label y
    y_ds = tf.data.Dataset.from_tensor_slices(y)
    images_label_ds = tf.data.Dataset.zip((ds_without_labels,y_ds))
    print(f"zipped_images_label_ds:{images_label_ds}")

    # print(ds_without_labels[0])  # 'TensorSliceDataset' object does not support indexing






In [20]:
build_dataset_by_random_sample()

ds_without_labels: <TensorSliceDataset shapes: (), types: tf.float64>
dataset with labels:<TensorSliceDataset shapes: ((), ()), types: (tf.float64, tf.float64)>
zipped_images_label_ds:<ZipDataset shapes: ((), ()), types: (tf.float64, tf.float64)>


In [21]:
def load_inner_dataset():
    """1.2  """
    pass

In [28]:
def dataloader_exsample():
    """2.0 Dataloader 
        use tf.data to shuffle, batch, repeat...
    """

    x = np.linspace(-1, 1, 10)
    y = x**2

    dataset  = tf.data.Dataset.from_tensor_slices((x,y))  
    # print(f"dataset with labels:{dataset}")

    BATCH_SIZE = 5
    dataloader = dataset.shuffle(buffer_size=len(x))
    dataloader = dataloader.repeat()
    dataloader = dataloader.batch(BATCH_SIZE)

    AUTOTUNE = tf.data.experimental.AUTOTUNE
    dataloader = dataloader.prefetch(buffer_size=AUTOTUNE)
    """"↑↑↑↑  Creates a `Dataset` that prefetches elements from this dataset.

                Most dataset input pipelines should end with a call to `prefetch`. This
                allows later elements to be prepared while the current element is being
                processed. This often improves latency and throughput, at the cost of
                using additional memory to store prefetched elements.
    """
    print(f"dataloader:{dataloader}")
    return dataloader


def dataloader_MNIST():
    """2.0 Dataloader 
        use tf.data to shuffle, batch, repeat...
    """


    (x_train, y_train), _ = tf.keras.datasets.mnist.load_data()
    dataset = tf.data.Dataset.from_tensor_slices(
        (x_train.reshape(60000, 784).astype("float32") / 255, y_train)
    )
    
    # print(f"dataset with labels:{dataset}")

    BATCH_SIZE = 1
    dataloader = dataset.shuffle(buffer_size=1024)
    dataloader = dataloader.repeat()
    dataloader = dataloader.batch(BATCH_SIZE)

    AUTOTUNE = tf.data.experimental.AUTOTUNE
    dataloader = dataloader.prefetch(buffer_size=AUTOTUNE)
    """"↑↑↑↑  Creates a `Dataset` that prefetches elements from this dataset.

                Most dataset input pipelines should end with a call to `prefetch`. This
                allows later elements to be prepared while the current element is being
                processed. This often improves latency and throughput, at the cost of
                using additional memory to store prefetched elements.
    """
    print(f"dataloader:{dataloader}")
    return dataloader

In [29]:
dataloader_exsample()
dataloader_MNIST()

dataloader:<PrefetchDataset shapes: ((None,), (None,)), types: (tf.float64, tf.float64)>
dataloader:<PrefetchDataset shapes: ((None, 784), (None,)), types: (tf.float32, tf.uint8)>


<PrefetchDataset shapes: ((None, 784), (None,)), types: (tf.float32, tf.uint8)>

In [42]:
def call_MLP():
    """3.1 Stack layers Model 
       inhiert from keras.layers.Layer!
    """
    from tensorflow import keras
    
    class Linear(keras.layers.Layer):
        """y = w.x + b"""

        def __init__(self, units=32, input_dim=32):
            super(Linear, self).__init__()
            w_init = tf.random_normal_initializer()
            self.w = tf.Variable(
                initial_value=w_init(shape=(input_dim, units), dtype="float32"),
                trainable=True,
            )
            b_init = tf.zeros_initializer()
            self.b = tf.Variable(
                initial_value=b_init(shape=(units,), dtype="float32"), trainable=True
            )

        def call(self, inputs):
            return tf.matmul(inputs, self.w) + self.b

    class MLP(keras.layers.Layer):
        def __init__(self):
            super(MLP, self).__init__()
            self.linear_1 = Linear(32)
            self.linear_2 = Linear(32)
            self.linear_3 = Linear(10)

        def call(self, inputs):
            x = self.linear_1(inputs)
            x = tf.nn.relu(x)
            x = self.linear_2(x)
            x = tf.nn.relu(x)
            return self.linear_3(x)
    
    mlp = MLP()
    print(f"MLP_model:{mlp}")
    # y = mlp(tf.ones(shape=(3, 64)))
    # assert len(mlp.weights) == 6
    return mlp

def model_Sequential_exsample():
    """3.2 Sequential model
       use keras.Sequential to build model
    """
    from tensorflow import keras
    classes = 10

    Sequential_model = keras.Sequential(
    [keras.layers.Dense(64, activation=tf.nn.relu), keras.layers.Dense(classes),]
    ) 

    print(f"Sequential_model:{Sequential_model}")

    return Sequential_model

def  functional_model():
    """3.3 build model functionally"""
    pass



In [41]:
call_MLP()
model_Sequential_exsample()

MLP_model:<__main__.call_MLP.<locals>.MLP object at 0x2b4079986b70>
Sequential_model:<tensorflow.python.keras.engine.sequential.Sequential object at 0x2b4079986ac8>


In [46]:
"""4.0 Optimzer """
# 1&2 Dataset and Dataloader
(x_train, y_train), _ = tf.keras.datasets.mnist.load_data()
dataset = tf.data.Dataset.from_tensor_slices(
    (x_train.reshape(60000, 784).astype("float32") / 255, y_train)
)
dataloader = dataset.shuffle(buffer_size=1024).batch(64)

# 3 model
mlp = call_MLP()

# 4 optimzer
optimizer = tf.keras.optimizers.SGD(learning_rate=1e-3)
loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=True)  # Whether `y_pred` is expected to be a logits tensor

#TODO ADD hypercolumn to LCN and loss


"""5.0 Train """
from tqdm import tqdm
epoch_num = 2
for epoch in range(epoch_num):
    with tqdm(dataloader) as train_bar:
        for step, (x,y) in enumerate(train_bar):
            
            with tf.GradientTape() as tape:  # in torch, without this GradientTape
                
                #! forward, edfault gradien zero?
                loss = loss_fn(y, mlp(x)) # more with zero_grad
                loss += sum(mlp.losses)# loss.backward() in torch
                gradient = tape.gradient(loss, mlp.trainable_weights) # in torch training paras setting in optim
            
            optimizer.apply_gradients(zip(gradient, mlp.trainable_weights))# in torch optimizer.step()
    
    if step % 100 == 0:
        print("Epoch",epoch, "Step:", step, "Loss:", float(loss))


# time = str(datetime.now()).split(" ")[0].replace("-", "_")
# torch.save(model.state_dict(), "model_{}.pth".format(time))

MLP_model:<__main__.call_MLP.<locals>.MLP object at 0x2b407af40160>


  0%|          | 0/938 [00:00<?, ?it/s]


InvalidArgumentError: Matrix size-incompatible: In[0]: [64,784], In[1]: [32,32] [Op:MatMul]